本家を見るに二つの解決があるが、ソートを考えたほうがバグも少ないものが書けるのでそういう風にする。

たぶんこれは二つの配列をマージするような関数を定義したほうがよいと思うので、それを定義する。

ソートされているので `(car lst) <= (car lst2)`は確実に保証されているとみなしてもいい。ではこの条件で、マージできない場合というのは`(second lst1) < (car lst2)`のとき。このときは`#f`を返し、マージ可能なら`(list (car lst) (max (second lst) (second lst2))`としてやるとよい。

（型が違うのはどうなのとも思うけど、OCamlじゃないから硬いことは考えない)

In [1]:
(define (merge-two-list lst lst2)
  (if (< (cadr lst) (car lst2)) #f
      (list (car lst) (max (cadr lst) (cadr lst2)))))

(require rackunit)
(check-equal? (merge-two-list (list 1 4) (list 4 5)) (list 1 5))
(check-equal? (merge-two-list (list 1 4) (list 5 6)) #f)

あとは再帰的にソートしたリストを巡っていく。ポイントとしては:

* **再帰関数の終了条件** は length(lst)が1か0のとき
  * 0のときはそのままresultを返す
  * 1のときはresultに残りを追加して返す
* **終了条件ではないとき** リストから二つ取ってきてマージできるかどうかを調べる
  * #fならそれをresultに追加して次のリストに向かう
  * リストが帰ってきたら、lstの先頭に追加する

In [2]:
(define (merge-two-list lst lst2)
  (if (< (cadr lst) (car lst2)) #f
      (list (car lst) (max (cadr lst) (cadr lst2)))))

(define (merge-list lst)
  (define (inner lst result)
      (cond [(= 0 (length lst)) (reverse result)]
            [(= 1 (length lst)) (reverse (append lst result))]
            [else
             (let ([try-merge (merge-two-list (car lst) (cadr lst))])
               (if try-merge (inner (append (list try-merge) (cddr lst)) result)
                   (inner (cdr lst) (append (list (car lst)) result))))]))
  (inner lst (list)))

(check-equal? (merge-list (list (list 1 3) (list 2 6) (list 8 10) (list 15 18))) (list (list 1 6) (list 8 10) (list 15 18)))
(check-equal? (merge-list (list (list 1 4) (list 4 5))) (list (list 1 5)))
